<a href="https://colab.research.google.com/github/bhar-gav/MDS-COLAB/blob/main/T5_Xsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
print(os.getcwd())
folder = "/content/drive/My Drive/Colab Notebooks/MDS"
os.chdir(folder)
print(os.getcwd())

Mounted at /content/drive
/content
/content/drive/My Drive/Colab Notebooks/MDS




---
preprocessing already applied :

```
# This code is already implemented on file
```






---
## **XSUM dataset**

(https://huggingface.co/datasets/EdinburghNLP/xsum)



---





In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

xsum = load_dataset("EdinburghNLP/xsum")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/6.24k [00:00<?, ?B/s]

xsum.py:   0%|          | 0.00/5.76k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/304M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/17.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

**Dataset Info**

In [ ]:
xsum

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id'],
        num_rows: 11334
    })
})



---



#**start point**






---
**DATA LOADER**

---


In [ ]:
!pip install transformers datasets torch rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f908e41a083721e8060d0b995014447b014da38059242d386915c2d4a462df77
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score




---



In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-small')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
xsum = xsum.map(preprocess_function, batched=True)
xsum

Map:   0%|          | 0/204045 [00:00<?, ? examples/s]

Map:   0%|          | 0/11332 [00:00<?, ? examples/s]

Map:   0%|          | 0/11334 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['document', 'summary', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 11334
    })
})

In [ ]:
xsum=xsum.remove_columns(['document', 'summary','id'])
xsum

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 204045
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11332
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11334
    })
})

In [ ]:
from transformers import T5ForConditionalGeneration, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

model=T5ForConditionalGeneration.from_pretrained('t5-small')


In [ ]:
# training arguments
training_arguments = Seq2SeqTrainingArguments(
            output_dir='./results',
            evaluation_strategy='epoch',

            save_strategy='epoch',  #save model state after each epoch (space consuming)
            #save_strategy='no',  #1. use to save model only once after finishing training (comment out above line in case)

            logging_dir='./logs',
            learning_rate=2e-5,
            per_device_train_batch_size=2,
            per_device_eval_batch_size=2,
            weight_decay=0.01,
            save_total_limit=4,
            num_train_epochs=2,
            # remove_unused_columns=False,
            fp16=True,
            )


trainer = Seq2SeqTrainer(
    model=model,
    args=training_arguments,
    train_dataset=xsum['train'],
    eval_dataset=xsum['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator
    )


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-ac52682c094f>:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

#trainer.save_model()  # 2.save model after completing training (use if (1.) is followed)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

**Evaluate**

In [ ]:
from transformers import DataCollatorForSeq2Seq
from torch.utils.data import DataLoader
import torch

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

def generate_summaries(test_dataset, model, tokenizer, batch_size=4, max_output_length=20):
    model.eval()
    all_summaries = []

    dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=data_collator)

    for batch in dataloader:
        input_ids = batch['input_ids'].to(model.device)
        attention_mask = batch['attention_mask'].to(model.device)

        summary_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_output_length,
            num_beams=4,
            length_penalty=2.0,
            early_stopping=True
        )


        summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
        all_summaries.extend(summaries)

    return all_summaries

generated_summaries = generate_summaries(xsum['test'], model, tokenizer)

for i in range(5):
    print(f"Generated Summary {i+1}: {generated_summaries[i]}")


In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.9 MB/s eta 0:00:00


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

def decode_labels(labels, tokenizer):
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return decoded_labels

generated_summaries = generate_summaries(xsum['test'], model, tokenizer)
reference_summaries = decode_labels(xsum['test']['labels'], tokenizer)


def calculate_rouge(generated_summaries, reference_summaries):
    results = rouge.compute(predictions=generated_summaries, references=reference_summaries)
    return results

rouge_results = calculate_rouge(generated_summaries, reference_summaries)


print("ROUGE Scores:")
print(rouge_results)



---
**su4 included**


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

def decode_labels(labels, tokenizer):
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return decoded_labels

generated_summaries = generate_summaries(xsum['test'], model, tokenizer)
reference_summaries = decode_labels(xsum['test']['labels'], tokenizer)


def calculate_rouge(generated_summaries, reference_summaries):
    results = rouge.compute(predictions=generated_summaries, references=reference_summaries, rouge_types=["rouge1", "rouge2", "rougeL", "rouge4"])
    return results

rouge_results = calculate_rouge(generated_summaries, reference_summaries)


print("ROUGE Scores:")
print(rouge_results)



---


Bert Model  : https://huggingface.co/google-t5/t5-small
---



END

---



---





---
**live test**
